In [1]:
import sys,os
import numpy as np
from matplotlib import pyplot as plt

# This is the standard path for SEACAS if Amanzi TPLS are built via
# bootstrap with --build-shared
try:
    import exodus
except ImportError:
    sys.path.append(os.path.join(os.environ['AMANZI_TPLS_DIR'],'SEACAS', 'lib'))
    import exodus

# This is the standard path for ATS's source directory    
try:
    import meshing_ats
except ImportError:
    sys.path.append(os.path.join(os.environ['ATS_SRC_DIR'],'tools','meshing','meshing_ats'))
    import meshing_ats

In [4]:
x = np.array([0,9.33,22.31,30.30,35.87,41.84,47.36,53.44,59.45,65.51,71.86,77.70,84.20,90.42,95.91,98.68,102.63,106.68,109.00])
z = np.array([892.00,891.71,891.41,891.24,891.17,891.02,890.93,890.81,890.58,890.41,890.33,890.17,889.93,889.77,889.58,889.53,889.41,889.25,889.00])

In [5]:
n = 10  # increase decrease horizontal coarseness
x_interp = np.linspace(x[0], x[-1], n)
z = np.interp(x_interp, x, z)
x = x_interp
dx = x[n-5] - x[n-6]
dx

np.float64(12.111111111111114)

In [6]:
m2 = meshing_ats.Mesh2D.from_Transect(x,z)
layer_types = []
layer_data = []
layer_ncells = []
layer_mat_ids = []
z = 0.0

In [7]:
# -- acrotelm layer --
#  top 6 cm (z_ac)
#  dz = 1 cm per grid cell
#  10 cells
layer_types.append("constant")
layer_data.append(0.06)
layer_ncells.append(6)
layer_mat_ids.append(1001)
z -= 0.06

# -- catotelm layer  --
#  z_ct =  27 cm
#  dz = 1 cm per grid cell
#  27 cells
layer_types.append("constant")
layer_data.append(0.27)
layer_ncells.append(27)
layer_mat_ids.append(1002)
z -= 0.27

# - mineral layer  --
#  z_mn =  60 cm
#  dz = 1 cm per grid cell
#  60 cells
layer_types.append("constant")
layer_data.append(0.6)
layer_ncells.append(60)
layer_mat_ids.append(1003)
z -= 0.6

# -- mineral layer part 2 --
#  50 cells
#  expanding dz, growing with depth
ncells = 50
dz = 0.02
for i in range(ncells):
    dz *= 1.05
    # print(dz)
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1003)
    z -= dz

# -- mineral layer part 3 --
#  keep expanding until we hit dz =  1.5 m
#  expanding dz, growing with depth
dz *= 1.05
while dz < 1.8:
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1003)
    z -= dz
    dz *= 1.05

# # -- mineral layer part 5 --
# # keep going for 1m cells until we hit the bottom of
# # the domain
layer_types.append("constant")
layer_data.append(40 + z) # depth of bottom of domain is 40 m
layer_ncells.append(int(round(layer_data[-1] / 2.0)))
layer_mat_ids.append(1003)


In [9]:
# Extrude the 2D model with this structure and write to file
m3 = meshing_ats.Mesh3D.extruded_Mesh2D(m2, layer_types, 
                                        layer_data, 
                                        layer_ncells, 
                                        layer_mat_ids)

m3.write_exodus("../data/transect_imnavait_grace_v2_coarse.exo")

Opening exodus file: ../data/transect_imnavait_grace_v2_coarse.exo
Closing exodus file: ../data/transect_imnavait_grace_v2_coarse.exo


In [11]:
exo_file = exodus.exodus("../data/transect_imnavait_grace_v2_coarse.exo", mode="r")
x, y, z = exo_file.get_coords()
xmin, xmax = min(x), max(x)
ymin, ymax = min(y), max(y)
zmin, zmax = min(z), max(z)

print(f"X Extent: {xmin} to {xmax}")
print(f"Y Extent: {ymin} to {ymax}")
print(f"Z Extent: {zmin} to {zmax}")

exo_file.close()

Opening exodus file: ../data/transect_imnavait_grace_v2_coarse.exo
X Extent: 0.0 to 109.0
Y Extent: 0.0 to 1.0
Z Extent: 849.0000000000003 to 892.0
Closing exodus file: ../data/transect_imnavait_grace_v2_coarse.exo
